# PICES Large Marine Ecosystem Regions
## Data acquisition, analysis, plotting & saving to facilitate IEA report
### Developed by: Chelle Gentemann (chellegentemann@gmail.com) & Marisol Garcia-Reyes (marisolgr@gmail.com)
***
***

# Instructions

## To configure:
### In the cell marked by `Configuration` specify: Region, variable, and date period
## To execute:
### Click on the top menu: `Cell` -> `Run All`  
***
***

# Regions
<table width='100%'><tr>
    <td width='10%'>
        <b>LME Number</b><br><br>
        11<br><br>12<br><br>13<br><br>14<br><br>15<br><br>16<br><br>17
        <br><br>18<br><br>19<br><br>20<br><br>21<br><br>22<br><br>23<br><br>24</td>
    <td width='20%' text-align='left'>
    <b> LME Name</b><br><br>
    California Current<br><br>
    Gulf of Alaska<br><br>
    East Bering Sea<br><br>
    North Bering Sea<br><br>
    Aleutian Islands<br><br>
    West Bering Sea<br><br>
    Sea of Okhotsk<br><br>
    Oyashio Current<br><br>
    Sea of Japan<br><br>
    Yellow Sea<br><br>
    East China Sea<br><br>
    Kuroshio Current<br><br>
    West North Pacific<br><br>
    East North Pacific<br></td>
    <td>
<img src='../figures/PICES_all_regions_mask_ne.png' alt='North Pacific LME Regions' width='750'>
    </td>
    </tr>
</table>


***

# Variables
## 1) SST: Sea Surface Temperature  ('1981-01-01' - present) 
## 2) Chl: Chlorphyll-a Concentration ('1997-01-01' - '2018-06-30')
## 3) Wind: Wind speed vector ('1997-01-01' - present)
## 4) Currents: Sea Surface Currents ('1992-01-01' - present)

***
***


###########################
#  ***  Configuration   ***
<br>
###########################

In [ ]:
#######################
#### Configuration ####
#######################

## Region to analyze ##
region = 16 # <<<----- Use number (11 to 24) based on table above

## Variable ##
## Select the variable to analyze from the list above
var = 'SST' # <<<----- Use short name given above. upper or lower case accepted.

## Period to analize ##
## Specify the period using the format: YYYY-MM-DD
## Data available specified above
## All data in monthly resolution
initial_date = '1998-01-01'
final_date   = '2017-08-31'

##############################
#### End of configuration ####
##############################

In [ ]:
def analyze_PICES_LME(region,var,initial_date,final_date):
    
    %matplotlib inline  
    import sys
    sys.path.append('./subroutines/')
    from pices import get_pices_data
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import warnings
    warnings.simplefilter('ignore') # filter some warning messages
    lmenames = ['California Current','Gulf of Alaska','East Bering Sea','North Bering Sea','Aleutian Islands','West Bering Sea',
            'Sea of Okhotsk','Oyashio Current','Sea of Japan','Yellow Sea','East China Sea','Kuroshio Current',
            'West North Pacific','East North Pacific']

    # check values

    # assign variables
    lmei = region
    lmename = lmenames[lmei-11]
    var = var.lower() # variable name in lower case

    # data aquisition
    dtmean, dtclim, dtanom = get_pices_data(var, lmei, initial_date, final_date)
    
    #print(dtmean)
    
    # extract and assign data
    # name in dataset
    allvars = dtmean.data_vars
    for key,val in allvars.items():
        nvar = key
    # short and long name for variable
    svar = var.upper()
    if svar=='SST':
        lvar = dtmean[nvar].attrs['long_name']
    elif svar=='CHL':
        lvar = dtmean.attrs['parameter']
        
    datasetname = dtmean.attrs['title']
        
    units = dtmean[nvar].attrs['units']    
    
    ## Data information
    print('Region = '+str(lmei)+' - '+lmename)
    print('Data = '+lvar)
    print('Units = '+units)
    print('Period = '+initial_date+' : '+final_date)
    print('Dataset = '+datasetname)
    
    # displaying time series data
    plt.figure(figsize=(10,4))
    plt.plot(dtmean.time,dtmean[nvar])
    plt.grid(True)
    plt.ylabel(svar+' ('+units+')')
    plt.title(lmename+' '+svar+' values')
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.show()

    # display climatology
    plt.figure(figsize=(5,4))
    plt.plot(dtclim.month, dtclim[nvar],'+-',color='k')
    plt.grid(True)
    plt.ylabel(svar+' ('+units+')')
    plt.xticks(range(1,13),['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],rotation=45)
    plt.title(lmename+' '+svar+' climatology')
    plt.show()
    
    ## display statistics
    print('\nMean '+svar+' value = ', round(dtmean[nvar].values.mean(),2),units)
    print('Median '+svar+' value = ', round(np.median(dtmean[nvar].values),2),units)
    print(svar+' Standard deviation = ', round(dtmean[nvar].values.std(),2),units)
    print('\n')
    print('Maximum '+svar+' value = ', round(dtmean[nvar].values.max(),2),units)
    print('Minimum '+svar+' value = ', round(dtmean[nvar].values.min(),2),units)
    print('\n')
    print('Maximum '+svar+' anomalies value = ', round(dtanom[nvar].values.max(),2),units)
    print('Minimum '+svar+' anomalies value = ', round(dtanom[nvar].values.min(),2),units)

    # display density plots
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    sns.distplot(dtmean[nvar], hist=True, kde=True, bins=30,
                 kde_kws={'linewidth': 2})
    plt.title(svar+' values density plot')
    plt.grid(True)
    plt.xlabel(svar+' ('+units+')')
    plt.subplot(1,2,2)
    sns.distplot(dtanom[nvar], hist=True, kde=True, bins=30,
                 kde_kws={'linewidth': 2})
    plt.title(svar+' anomalies density plot')
    plt.grid(True)
    plt.xlabel(svar+' ('+units+')')
    plt.show()
    
    # display anomalies
    plt.figure(figsize=(12,4),dpi=200)
    p=dtanom.where(dtanom>=0)
    n=dtanom.where(dtanom<0)
    plt.bar(p.time.values,p[nvar], width=30, color='darkred',alpha=0.8, edgecolor=None,zorder=2)
    plt.bar(n.time.values,n[nvar], width=30, color='darkblue',alpha=0.8, edgecolor=None,zorder=3)
    plt.grid(True,zorder=1)
    plt.ylabel(svar+' ('+units+')')
    plt.title(lmename+' '+svar+' anomalies')
    plt.autoscale(enable=True, axis='x', tight=True)
    #plt.savefig('GoA_SST.png')
    plt.show()
    print('Anomalies calculated based on entire data period')

In [ ]:
#### Do not modify ####

analyze_PICES_LME(region, var, initial_date, final_date)

#### End of script ####